In [19]:
flower_mapping = {'daisy':0, 'dandelion':1, 'rose':2, 'sunflower':3, 'tulip':4} 

train_data_path = './Day_101/ml100marathon-final-exam/image_data/train'

### Refer to https://github.com/lucasdupin/ml-image-scaling/blob/master/1_preprocessing.ipynb

In [20]:
import numpy as np
import matplotlib.pyplot as plt  

from PIL import Image # Make sure you have Pillow! PIL may import but WILL raise exceptions

%matplotlib inline

In [37]:
import os
import sys

original_size = (150, 150)
small_size = (100, 100)

def image_cache_root_path(image_folder):
    return os.path.join(image_folder, '.cache')

def image_cache_folder_path(image_folder, resolution):
    return os.path.join(image_cache_root_path(image_folder), resolution)
    
def image_cache_file_path(image_folder, image_file, resolution):
    return os.path.join(image_cache_folder_path(image_folder, resolution), image_file)
    

def process_image(folder, image_path, image_cache_high, image_cache_low):
    """ Resizes an image, creating a big and a reduced version """
    
    # Skip files that already were generated or directories
    if os.path.exists(image_cache_high) and os.path.exists(image_cache_low):
        return
    
    with Image.open(image_path) as original:
  
        # Ignore images smaller than our target size
        if original.size[0] < original_size[0] or original.size[1] < original_size[1]:
            # print("image %s with size %s is too small, skipping..." % (image_path, original.size))
            return

        #print("resizing and cropping: %s" % image_path)
        center = (original.size[0]/2, original.size[1]/2)
        size = original_size[0]
        cropped = original.crop((center[0] - size/2, 
                                 center[1] - size/2, 
                                 center[0] + size/2,
                                 center[1] + size/2))
        # High-res version - labels
        cropped.save(image_cache_high)
        # Generate low-res version of the image - train data
        small = cropped.resize(small_size, Image.ANTIALIAS)
        small.save(image_cache_low)


def process_imagefolder_recursive(folder):
    
    print("Processing folder %s" % (folder))
    #print(".", end="")
    
    image_cache_root = image_cache_root_path(folder)
    if not os.path.exists(image_cache_root):
        os.mkdir(image_cache_root)

    image_cache_folder_high = image_cache_folder_path(folder, 'high')
    if not os.path.exists(image_cache_folder_high):
        os.mkdir(image_cache_folder_high)

    image_cache_folder_low = image_cache_folder_path(folder, 'low')
    if not os.path.exists(image_cache_folder_low):
        os.mkdir(image_cache_folder_low)

    # Get list of all image folders
    for d in sorted(os.listdir(folder)):
        
        # Ignore cache itself
        if d.startswith("."):
            continue
        
        # Recursive to folder
        if os.path.isdir(os.path.join(folder, d)):
            process_imagefolder_recursive(os.path.join(folder, d))
            continue

        # Check picture only
        if d.lower().endswith(('.jpg', 'png')):
            image_path = os.path.join(folder, d)
            image_cache_high = image_cache_file_path(folder, d, 'high')
            image_cache_low = image_cache_file_path(folder, d, 'low')
            process_image(folder, image_path, image_cache_high, image_cache_low)
        
process_imagefolder_recursive('.\Day_101\ml100marathon-final-exam\image_data')
        
print("Done!")

Processing folder .\Day_101\ml100marathon-final-exam\image_data
Processing folder .\Day_101\ml100marathon-final-exam\image_data\test
Processing folder .\Day_101\ml100marathon-final-exam\image_data\train
Processing folder .\Day_101\ml100marathon-final-exam\image_data\train\daisy
Processing folder .\Day_101\ml100marathon-final-exam\image_data\train\dandelion
Processing folder .\Day_101\ml100marathon-final-exam\image_data\train\rose
Processing folder .\Day_101\ml100marathon-final-exam\image_data\train\sunflower
Processing folder .\Day_101\ml100marathon-final-exam\image_data\train\tulip
Done!


In [ ]:
import imageio

In [95]:
# Read image file to memory as RGB array
def load_image_to_rgb(image_path):
    im = imageio.imread(image_path)
    # print(im.shape)
    return im


In [96]:
import numpy as np

In [99]:
def load_image_and_label(folder):
    count = 0
    image_array = np.empty((0, 100, 100, 3), int)
    print(f'image_array.shape = {image_array.shape}')
    for d in os.listdir(folder):
        if not d.lower().endswith(('.jpg', 'png')):
            continue
        im = load_image_to_rgb(os.path.join(folder, d))
        if im is None:
            print(f'Error when loading image:{os.path.join(folder, d)}')
            continue
        # img_array[count] = im
        #print(f'im.shape = {im.shape}')
        a = np.empty((1, 100, 100, 3), int)
        a[0] = im
        image_array = np.append(image_array, a, axis=0)
        # print(f'image_array.shape = {image_array.shape}')
        
    return image_array

In [100]:

image_cache_low = image_cache_folder_path(os.path.join('.\Day_101\ml100marathon-final-exam\image_data', 'train', 'daisy'), 'low')

x_img_train = load_image_and_label(image_cache_low)
print(np.asarray(x_img_train).shape)
y_label_train = []
x_img_test = []
y_label_test = []

image_array.shape = (0, 100, 100, 3)
(489, 100, 100, 3)


In [102]:
x_img_train

array([[[[100, 153, 223],
         [ 35, 132, 213],
         [  0, 138, 237],
         ...,
         [  1, 129, 238],
         [  1, 129, 238],
         [  1, 129, 238]],

        [[107, 160, 228],
         [ 43, 138, 218],
         [  0, 140, 238],
         ...,
         [  1, 129, 238],
         [  1, 129, 238],
         [  1, 129, 238]],

        [[118, 169, 234],
         [ 56, 148, 225],
         [  0, 143, 239],
         ...,
         [  1, 129, 238],
         [  1, 129, 238],
         [  1, 129, 238]],

        ...,

        [[248, 239, 232],
         [247, 238, 231],
         [244, 237, 229],
         ...,
         [  2, 113, 247],
         [  3, 114, 248],
         [  4, 115, 249]],

        [[246, 237, 230],
         [245, 236, 229],
         [242, 235, 227],
         ...,
         [  1, 112, 246],
         [  1, 112, 246],
         [  1, 112, 246]],

        [[246, 237, 230],
         [244, 235, 228],
         [241, 234, 226],
         ...,
         [  3, 113, 250],
        